# LSSTCam pointing errors
# Looking at the results of homing the mount.

Craig Lage - 18-Apr-25

In [ ]:
import os, sys
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

## Get the MTMount state changes

In [ ]:
startDay = 20250806
endDay = 20250828
dayObs = startDay
homeDict = {}
while dayObs <= endDay:
    try:
        start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
        end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
        
        homes = getEfdData(
        client,
        "lsst.sal.MTMount.command_homeBothAxes",
        begin=start,
        end=end
    )
        print(f"There were {len(homes)} home events on {dayObs}")
        for t in homes.index:
            t = Time(t)
            azel_start = t - TimeDelta(1.0, format='sec')
            azel_end = t + TimeDelta(1.0, format='sec')
            az = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=['actualPosition'],
                begin=azel_start,
                end=azel_end,
            )
            el = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=['actualPosition'],
                begin=azel_start,
                end=azel_end,
            )
            print(f"\t{t.isot}, az = {np.median(az['actualPosition'].values):.2f}, el = {np.median(el['actualPosition'].values):.2f}")
                        
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue

In [ ]:
t = Time(homes.index[0])
azel_start = t - TimeDelta(1.0, format='sec')
azel_end = t + TimeDelta(1.0, format='sec')



az = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    topic='actualPosition',
    begin=azel_start,
    end=azel_end,
)
